In [51]:
import tensorflow as tf

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
from textblob import TextBlob
from collections import Counter
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,GRU,LSTM
from tensorflow.keras.models import Model
import keras.backend as K
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
from tensorflow.keras import initializers, regularizers, constraints
import six
from tensorflow.keras.utils import deserialize_keras_object
from joblib import dump, load

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,GRU,Embedding,Flatten
from tensorflow.keras.models import Model
import keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, Dropout, GRU, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.layers import Layer
from tensorflow.keras import layers
import tensorflow as tf
 
from tensorflow.keras.layers import concatenate, Lambda


In [18]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        
      super().__init__()

      self.e_embed = Embedding(input_dim=inp_vocab_size, output_dim=embedding_size, input_length= input_length)
      self.lstm_size = lstm_size
      
      self.e_lstm = LSTM(lstm_size, return_sequences=True, return_state=True)


    def call(self,input_sequence,states):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- encoder_output, last time step's hidden and cell state
        '''
        embedding = self.e_embed(input_sequence)
        # print(embedding.shape)
        # output_state, enc_h, enc_c = self.e_lstm( embedding, initial_state = states)
        output_state, enc_h, enc_c = self.e_lstm( embedding)

        return output_state, enc_h, enc_c

      

    
    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      return [tf.zeros((batch_size, self.lstm_size)), tf.zeros((batch_size, self.lstm_size))]



In [19]:
class Attention(tf.keras.layers.Layer):
  '''
    Class the calculates score based on the scoring_function using Bahdanu attention mechanism.
  '''
  def __init__(self,scoring_function, att_units):

    super(Attention, self).__init__()

    self.scoring_function = scoring_function


    # Please go through the reference notebook and research paper to complete the scoring functions

    if self.scoring_function=='dot':
      # Intialize variables needed for Dot score function here
      self.dot= tf.keras.layers.Dot(axes = (1,2))      
      pass
    elif scoring_function == 'general':
      # Intialize variables needed for General score function here
      self.w= Dense(att_units)
      self.dot = tf.keras.layers.Dot(axes = (1,2))
      pass
    elif scoring_function == 'concat':
      # Intialize variables needed for Concat score function here
      self.w1= Dense(att_units)
      self.w2= Dense(att_units)
      self.v= Dense(1)
      pass
  
  
  def call(self,decoder_hidden_state,encoder_output):
    '''
      Attention mechanism takes two inputs current step -- decoder_hidden_state and all the encoder_outputs.
      * Based on the scoring function we will find the score or similarity between decoder_hidden_state and encoder_output.
        Multiply the score function with your encoder_outputs to get the context vector.
        Function returns context vector and attention weights(softmax - scores)
    '''
    decoder_hidden_state = tf.expand_dims(decoder_hidden_state, 1)
    if self.scoring_function == 'dot':
        # Implement Dot score function here
        score = tf.transpose(self.dot([tf.transpose(decoder_hidden_state, (0,2,1)), encoder_output]), (0,2,1))
        # print("In Dot")

        pass
    elif self.scoring_function == 'general':
        # Implement General score function here
        # print(encoder_output.shape)
        mulpy = self.w(encoder_output)
        # print(mulpy.shape)
        score = tf.transpose(self.dot([tf.transpose(decoder_hidden_state, (0, 2, 1)), mulpy]), (0, 2,1),)

        pass
    elif self.scoring_function == 'concat':
        inte = self.w1(decoder_hidden_state) + self.w2(encoder_output)
        tan = tf.nn.tanh(inte)
        score = self.v(tan)

    att_weights = tf.nn.softmax(score, axis =1)
    context_vector = att_weights * encoder_output
    context_vector = tf.reduce_sum( context_vector, axis=1)
    return context_vector, att_weights

        
    
    

In [20]:
class OneStepDecoder(tf.keras.Model):
  def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):

      # Initialize decoder embedding layer, LSTM and any other objects needed
      super(OneStepDecoder, self).__init__()
      self.dec_embed = Embedding(input_dim = tar_vocab_size, output_dim = embedding_dim)
      self.lstm = LSTM(dec_units, return_sequences = True, return_state = True)
      self.attention = Attention(scoring_function = score_fun, att_units = att_units)
      self.den = Dense(tar_vocab_size)


  def call(self,input_to_decoder, encoder_output, state_h,state_c):
    '''
        One step decoder mechanisim step by step:
      A. Pass the input_to_decoder to the embedding layer and then get the output(batch_size,1,embedding_dim)
      B. Using the encoder_output and decoder hidden state, compute the context vector.
      C. Concat the context vector with the step A output
      D. Pass the Step-C output to LSTM/GRU and get the decoder output and states(hidden and cell state)
      E. Pass the decoder output to dense layer(vocab size) and store the result into output.
      F. Return the states from step D, output from Step E, attention weights from Step -B
    '''
    embd = self.dec_embed(input_to_decoder)
    context_vec, attention_weights = self.attention( state_h,  encoder_output)    

    f_inp = tf.concat([tf.expand_dims(context_vec, 1), embd], axis = -1)
    # print(f_inp.shape)
    out, dec_h, dec_c = self.lstm(f_inp, [state_h, state_c])
    out = tf.reshape( out, (-1, out.shape[2]))
    output = self.den(out)

    return output, dec_h, dec_c, attention_weights, context_vec


In [21]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
      #Intialize necessary variables and create an object from the class onestepdecoder
      super(Decoder, self).__init__()
      self.input_length = input_length
      self.out_vocab_size = out_vocab_size
      self.oneStepDecoder = OneStepDecoder(out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units)
      self.out_vocab_size = out_vocab_size
        
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state ):

        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        #Create a tensor array as shown in the reference notebook
        
        #Iterate till the length of the decoder input
            # Call onestepdecoder for each token in decoder_input
            # Store the output in tensorarray
        # Return the tensor array
        # outputs = tf.TensorArray(dtype =  tf.float32, size= input_to_decoder.shape[1])
        outputs = tf.TensorArray(dtype =  tf.float32, size= tf.shape(input_to_decoder)[1])

        
        for timestep in range(tf.shape(input_to_decoder)[1]):

            output, decoder_hidden_state, decoder_cell_state, _, _ = self.oneStepDecoder(input_to_decoder[:, timestep:timestep+1],
                                                                                          encoder_output,decoder_hidden_state,decoder_cell_state)                                                                                            
                                                                                             
                                                                                             
            # Store the output in tensorarray
            outputs = outputs.write(timestep, output)
        # Return the tensor array
        outputs = tf.transpose(outputs.stack(), (1, 0, 2))
        return outputs
        
        
    

In [22]:
class encoder_decoder(tf.keras.Model):
  def __init__(self,inp_vocab_size, out_vocab_size,input_length, enc_units,embedding_dim, dec_units, max_len, score_fun, att_units, batch_size):
    #Intialize objects from encoder decoder
    super(encoder_decoder, self).__init__()
    self.encoder = Encoder(inp_vocab_size= inp_vocab_size +1, embedding_size=embedding_dim, lstm_size= att_units,input_length= max_len)
    self.decoder = Decoder(out_vocab_size +1, embedding_dim, input_length, dec_units ,score_fun ,att_units)
    self.batch_size = batch_size

  
  def call(self,data):
    #Intialize encoder states, Pass the encoder_sequence to the embedding layer
    # Decoder initial states are encoder final states, Initialize it accordingly
    # Pass the decoder sequence,encoder_output,decoder states to Decoder
    # return the decoder output
    e_inp, d_inp = data[0], data[1]
    # print(data[0].shape)
    # print(data[1].shape)


    initial_state = self.encoder.initialize_states(self.batch_size)

    e_output, enc_h, enc_c = self.encoder(e_inp,initial_state)
    outputs = tf.TensorArray(dtype = tf.float32, size= 20)
        
    dec_h = enc_h
    dec_c = enc_c
    # print(dec_h.shape)
    output=self.decoder(d_inp,e_output, dec_h, dec_c)
  
    return output




In [75]:
tokenizer = Tokenizer()

In [81]:
with open('/Users/sayedraheel/Desktop/NLP/Sentence_correction/tokenizer.word_index.json', 'r') as json_file:
    loaded_word_index = json.load(json_file)

# Create a new tokenizer and set its word_index to the loaded word_index
tokenizer = Tokenizer()
tokenizer.word_index = loaded_word_index

In [82]:
with open('/Users/sayedraheel/Desktop/NLP/Sentence_correction/tokenizer_e.word_index.json', 'r') as json_file:
    loaded_word_index1 = json.load(json_file)

# Create a new tokenizer and set its word_index to the loaded word_index
tokenizer_e = Tokenizer()
tokenizer_e.word_index = loaded_word_index1

In [83]:
encoder_vocab_size=len(tokenizer.word_index.keys())
print(encoder_vocab_size)
decoder_vocab_size=len(tokenizer_e.word_index.keys())
print(decoder_vocab_size)

7815
5464


In [80]:
loaded_tokenizer_e.word_index

{'<start>': 1,
 '<end>': 2,
 'I': 3,
 'you': 4,
 'to': 5,
 'the': 6,
 'are': 7,
 'is': 8,
 'a': 9,
 'for': 10,
 'and': 11,
 'at': 12,
 "I'm": 13,
 'your': 14,
 'my': 15,
 'You': 16,
 'go': 17,
 'have': 18,
 'me': 19,
 'in': 20,
 'of': 21,
 'can': 22,
 'be': 23,
 'want': 24,
 'on': 25,
 'not': 26,
 'will': 27,
 "don't": 28,
 'with': 29,
 'or': 30,
 'So': 31,
 'it': 32,
 'am': 33,
 'going': 34,
 'do': 35,
 'Haha.': 36,
 'that': 37,
 'so': 38,
 'got': 39,
 'just': 40,
 'then': 41,
 'think': 42,
 'Then': 43,
 'know': 44,
 'Hey,': 45,
 'you.': 46,
 'we': 47,
 'What': 48,
 'But': 49,
 'if': 50,
 'time': 51,
 'meet': 52,
 'what': 53,
 'very': 54,
 'already.': 55,
 'How': 56,
 'this': 57,
 "I'll": 58,
 'see': 59,
 'you?': 60,
 'still': 61,
 'all': 62,
 'come': 63,
 'Are': 64,
 'about': 65,
 'like': 66,
 'get': 67,
 'from': 68,
 'now.': 69,
 'up': 70,
 'out': 71,
 'need': 72,
 'but': 73,
 'Can': 74,
 'me.': 75,
 'Haha,': 76,
 'was': 77,
 'when': 78,
 'how': 79,
 'Ok.': 80,
 'Yes.': 81,
 'My': 8

In [85]:
tokenizer.word_index=loaded_tokenizer.word_index.keys()
tokenizer_e.word_index=loaded_tokenizer_e.word_index

In [48]:
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense,Input,GRU,Embedding,Flatten

In [49]:
encoder_vocab_size=7815
decoder_vocab_size=5464
lstm_size = 100
embedding_dim =128
att_units = 100
dec_units=100
batch_size = 64

In [56]:
loaded_model = encoder_decoder(encoder_vocab_size,decoder_vocab_size,39,lstm_size,embedding_dim,att_units, 40,'concat',dec_units,batch_size)

In [57]:
loaded_model

In [59]:
dummy_encoder_input = tf.constant([[1, 2, 3, 4, 5]])  # Replace with your own input
dummy_decoder_input = tf.constant([[6, 7, 8, 9, 10]]) 

In [60]:
loaded_model([dummy_encoder_input, dummy_decoder_input])

<tf.Tensor: shape=(1, 5, 5465), dtype=float32, numpy=
array([[[-2.2220929e-04,  9.5734210e-04,  1.8716115e-03, ...,
         -8.7533968e-05, -3.0937325e-03,  3.0638601e-04],
        [-3.4844616e-04, -2.8433601e-04,  2.7361875e-03, ...,
         -1.9186018e-03, -2.4391573e-03,  2.0527642e-03],
        [ 8.2773832e-04, -3.2353846e-04,  2.5894430e-03, ...,
         -2.1019522e-03, -1.6265627e-03,  2.2080115e-03],
        [-3.5861123e-04,  5.5544788e-04,  1.9033194e-03, ...,
         -5.2289100e-04, -1.5952929e-03, -9.3154743e-04],
        [-1.1149305e-03, -4.8895524e-04,  1.7227859e-03, ...,
         -1.2337214e-03, -1.9210590e-03, -1.6469908e-03]]], dtype=float32)>

In [62]:
loaded_model.load_weights('/Users/sayedraheel/Desktop/NLP/Sentence_correction/encoder_decoder_weights.h5')

In [64]:
loaded_model.summary

<bound method Model.summary of <__main__.encoder_decoder object at 0x285864af0>>

In [65]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

def evaluate(sentence):
  max_length_targ = 40
  max_length_inp  = 39
  
  attention_plot = np.zeros((max_length_targ, max_length_inp))
  # sentence = preprocess(sentence)
  # sentence = sentence.strip()
  # print(sentence)
  #pdb.set_trace()
  inputs = [] 
  # for word in sentence.split():
  #     print(word)
  #     inputs.append(tokenizer.word_index[word])
  inputs = tokenizer.texts_to_sequences([sentence])
  inputs =  tf.keras.preprocessing.sequence.pad_sequences(inputs,maxlen=max_length_inp,padding='post') 
  inputs = tf.convert_to_tensor(inputs)
  result = '' 

  initial_state=model2.layers[0].initialize_states(batch_size=1)
  encoder_outputs, state_h,state_c = model2.layers[0](inputs,initial_state)   
  dec_input = tf.expand_dims([tokenizer_e.word_index['<start>']], 0)
  # dec_input=tf.expand_dims([tokenizer_e.word_index['\t']],0)


  for i in range(max_length_targ):
    Output,state_h,state_c,att_weights,_ = model2.layers[1].oneStepDecoder(dec_input,encoder_outputs,state_h,state_c)
    #Beam Search Decoder
    Result_beam_list=beam_search_decoder(Output,k=1)
    Result_beam=Result_beam_list[0][0]

    attention_weights = tf.reshape(att_weights, (-1, ))
    attention_plot[i]  = attention_weights.numpy()

    predicted_id = tf.argmax(Output[0]).numpy()
  
    result += tokenizer_e.index_word[Result_beam[0]] + ' '
    if tokenizer_e.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)
  return result, sentence, attention_plot

In [66]:

def predict(input_sentence):

  '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <\t> as input to decoder. and encoder final states as input_states to onestepdecoder.
  D. till we reach max_length of decoder or till the model predicted character <\n>:
         predictions, input_states, attention_weights = model.layers[1].onestepdecoder(input_to_decoder, encoder_output, input_states)
         And get the character using the tokenizer(character index) and then store it in a string.
  E. Return the predicted sentence

  '''


  input_sequence=tokenizer.texts_to_sequences([input_sentence])
  

  inputs=pad_sequences(input_sequence,maxlen=39,padding='post')
  inputs=tf.convert_to_tensor(inputs)
  result=''
  units=100
  hidden=[tf.zeros((1,units))]
  encoder_output,hidden_state,cell_state=model2.layers[0](inputs,hidden)
  dec_hidden=hidden_state
  dec_input=tf.expand_dims([tokenizer_e.word_index['<start>']],0)
  for t in range(40):
      predictions,dec_hidden,cell_state,attention_weights,context_vector=model2.layers[1].oneStepDecoder(dec_input,encoder_output,dec_hidden,cell_state)

      predicted_id=tf.argmax(predictions[0]).numpy()
      result+=tokenizer_e.index_word[predicted_id] + ' '
      if tokenizer_e.word_index['<end>']==predicted_id:
          return result
      dec_input= tf.expand_dims([predicted_id],0)
  return result

In [67]:
text= " hi am onkar iz tryinggg to "

In [69]:
predict(text.values[index])

AttributeError: 'str' object has no attribute 'values'

In [73]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [84]:
import numpy as np

def translate_and_evaluate(input_text):
    # Tokenize the input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    inputs = tf.keras.preprocessing.sequence.pad_sequences(input_sequence, maxlen=39, padding='post')
    inputs = tf.convert_to_tensor(inputs)

    # Translate the input text using the predict function
    translation = predict(inputs)

    # Evaluate the translation using the evaluate function
    translation_result, original_sentence, attention_plot = evaluate(translation)
    
    return translation_result, original_sentence, attention_plot

# Replace 'your_input_text' with the actual text you want to translate and evaluate
input_text = "your_input_text"

# Translate and evaluate the input text
translation_result, original_sentence, attention_plot = translate_and_evaluate(input_text)

# Print the translation result and attention plot
print("Original Sentence:", original_sentence)
print("Translation Result:", translation_result)


AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'lower'